In [1]:
#Importing the dataset
import pandas as pd

df1=pd.read_csv('../input/toxic-comments/train.csv').fillna(' ')

In [2]:
df2=pd.read_csv('../input/toxic-comments/test.csv').fillna(' ')

In [4]:
df=pd.concat([df1,df2])

In [5]:
df1.head(10)

In [6]:
df.shape

In [3]:
#Removing special characters from comment_text
import re

def clean_text(text):
    return re.sub(r'\W+',' ', text)

In [8]:
df['comment_text_clean']=df['comment_text']
df['comment_text_clean']=df['comment_text_clean'].apply(clean_text)

In [4]:
#Train data
df1['comment_text_clean']=df1['comment_text']
df1['comment_text_clean']=df1['comment_text_clean'].apply(clean_text)
#Test data
df2['comment_text_clean']=df2['comment_text']
df2['comment_text_clean']=df2['comment_text_clean'].apply(clean_text)

In [5]:
#FormattingText
def changecase(text):
    return text.lower()

In [11]:
df['comment_text_clean']=df['comment_text']
df['comment_text_clean']=df['comment_text_clean'].apply(changecase)

In [6]:
#Train data
df1['comment_text_clean']=df1['comment_text']
df1['comment_text_clean']=df1['comment_text_clean'].apply(changecase)
#Test data
df2['comment_text_clean']=df2['comment_text']
df2['comment_text_clean']=df2['comment_text_clean'].apply(changecase)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer=CountVectorizer(stop_words='english', min_df=0.0001,analyzer='word',max_features=10000,ngram_range=(1,2))
#X_train=vectorizer.fit(df['comment_text_clean'])
X_train=vectorizer.fit_transform(df1['comment_text_clean'])
#X_train=vectorizer.transform(df1['comment_text_clean'])
X_test=vectorizer.transform(df2['comment_text_clean'])

In [14]:
#Check if the dataset is balanced -
len(df[df['identity_hate']==1])

In [15]:
df.describe()

In [9]:
target = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
y_train=df1[target]
y_train

In [17]:
#Building SVC model
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split

X_train, X_calib, y_train, y_calib=train_test_split(X_train,y_train,test_size=0.2)

svm_clf_m1=LinearSVC(random_state=42)
cali_clf_m1=CalibratedClassifierCV(base_estimator=svm_clf_m1,cv='prefit')
result=pd.DataFrame()
result_cali=pd.DataFrame()

for labels in target:
    svm_clf_m1.fit(X_train,y_train[labels])
    cali_clf_m1.fit(X_calib,y_calib[labels])
    result[labels]=svm_clf_m1.predict(X_train)
    result_cali[labels]=cali_clf_m1.predict(X_calib)

result

In [18]:
result_cali['toxic'].value_counts()

In [19]:
len(result[result['toxic']==1])

In [7]:
#Using TFIDF as the feature 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english',min_df=0.0001,analyzer='word',max_features=10000,ngram_range=(1,2))
#X_train_2=tfidf_vectorizer.fit(df['comment_text_clean'])
X_train_2=tfidf_vectorizer.fit_transform(df1['comment_text_clean'])
#X_train_2=tfidf_vectorizer.transform(df1['comment_text_clean'])
X_test_2=tfidf_vectorizer.transform(df2['comment_text_clean'])

In [21]:
X_train_2.shape

In [11]:
from sklearn.model_selection import train_test_split
y_train_2 = df1[target]
X_train_2, X_calib_2, y_train_2, y_calib_2=train_test_split(X_train_2,y_train_2,test_size=0.2,random_state=42)

In [23]:
from sklearn.metrics import accuracy_score
result_1=pd.DataFrame()
result_cali_m2=pd.DataFrame()
svm_clf_m2=LinearSVC(random_state=42)
cali_clf_m2=CalibratedClassifierCV(base_estimator=svm_clf_m2,cv='prefit')

for labels in target:
    svm_clf_m2.fit(X_train_2,y_train_2[labels])
    cali_clf_m2.fit(X_calib_2,y_calib_2[labels])
    #result_1[labels]=svm_clf_m2.predict(X_calib_2)
    result_cali_m2[labels]=cali_clf_m2.predict(X_calib_2)
    accuracy_sc=accuracy_score(y_calib_2[labels],result_cali_m2[labels])
    print(labels,'-',accuracy_sc)

In [24]:
#len(result_1[result_1['toxic']==1])

In [25]:
from scipy.sparse import csr_matrix
X_test_2=csr_matrix.toarray(X_test_2)

In [26]:
X_test_2

In [27]:
#Running predict function on both models
import numpy as np
#result_test_m1=pd.DataFrame()
#result_test_m2=pd.DataFrame()
test_val=0
X_test=np.array(X_test)


#for labels in target:
result_test_m1=cali_clf_m1.predict_proba(X_test_2)
result_test_m2=cali_clf_m2.predict_proba(X_test_2)

print(result_test_m1.shape)

In [28]:
type(result_test_m1)

In [29]:
result_test_m1X=pd.DataFrame()
result_test_m2X=pd.DataFrame()

result_test_m1X['id']=df2['id']
result_test_m2X['id']=df2['id']

for labels in target:
    result_test_m1=cali_clf_m1.predict_proba(X_test_2)[:,1]
    result_test_m1X[labels]=pd.DataFrame(result_test_m1)
    result_test_m2=cali_clf_m1.predict_proba(X_test_2)[:,1]
    result_test_m2X[labels]=pd.DataFrame(result_test_m2)

In [30]:
print(result_test_m1X)

In [31]:
print(result_test_m2X)

In [32]:
#Creating Logistic Regression model
#from sklearn.linear_model import LogisticRegression
#log_clf=LogisticRegression()
#result_log=pd.DataFrame()

#for labels in target:
#    log_clf.fit(X_train,df1[labels])
#    result_log[labels]=log_clf.predict(X_train)
    
#result_log

In [33]:
#result_log['toxic'].value_counts()

In [26]:
#RandomForest model
from sklearn.ensemble import RandomForestClassifier
forest_clf=RandomForestClassifier(n_estimators=50,max_depth=200,random_state=42)
result_forest=pd.DataFrame()
X_train_forest=tfidf_vectorizer.fit_transform(df1['comment_text_clean'])

for labels in target:
    forest_clf.fit(X_train_forest,df1[labels])
    result_forest[labels]=forest_clf.predict(X_train_forest)
    
result_forest

In [27]:
result_forest['toxic'].value_counts()

In [36]:
#Evaluating Logistic and RandomForest model on Test data
#result_test_log=pd.DataFrame()
#result_test_forest=pd.DataFrame()

#for labels in target:
#    result_test_log[labels]=log_clf.predict(X_test)
#    result_test_forest[labels]=forest_clf.predict(X_test)
#    
#print(len(result_test_log[result_test_log['toxic']==1]),len(result_test_forest[result_test_forest['toxic']==1]))

In [28]:
result_test_forestX = pd.DataFrame()
result_test_forestY = pd.DataFrame()

result_test_forestX['id']=df2['id']
result_test_forestY['id']=df2['id']

for labels in target:
    result_test_forest=forest_clf.predict_proba(X_test_2)[:,1]
    result_test_forestX[labels]=pd.DataFrame(result_test_forest)
    result_test_forestY[labels]=forest_clf.predict(X_test_2)
    
result_test_forestX

In [29]:
result_test_forestY

In [30]:
result_test_forestY['toxic'].value_counts()

In [38]:
#Naive Bayes Model - Works on the principal that 2 events occuring are independent of each other - Same as NB of probability 
from sklearn.naive_bayes import MultinomialNB

result_NB=pd.DataFrame()
NB_clf=MultinomialNB()

for labels in target:
    NB_clf.fit(X_train_forest,df1[labels])
    result_NB[labels]=NB_clf.predict(X_train_forest)
    
result_NB

In [39]:
result_NB['toxic'].value_counts()

In [40]:
result_test_NBX = pd.DataFrame()
result_test_NBY = pd.DataFrame()

result_test_NBX['id']=df2['id']
result_test_NBY['id']=df2['id']

for labels in target:
    result_test_NB=forest_clf.predict_proba(X_test_2)[:,1]
    result_test_NBX[labels]=pd.DataFrame(result_test_NB)
    result_test_NBY[labels]=forest_clf.predict(X_test_2)
    
result_test_NBX

In [41]:
result_test_NBY['toxic'].value_counts()